# Pembangunan Model Akustik Bertahap - Sistem Pengenal Emosi Cakapan

## Library Preparation

In [1]:
!pip install arff
!pip install imbalanced-learn

  Created wheel for arff: filename=arff-0.9-cp37-none-any.whl size=4970 sha256=5cb346395253c639eaaf14d0a23957a23c42bca688f3866401b719b59b070cde
  Stored in directory: /root/.cache/pip/wheels/04/d0/70/2c73afedd3ac25c6085b528742c69b9587cbdfa67e5194583b
Successfully built arff


In [2]:
import arff
import joblib
from io import StringIO
from google.colab import drive
from google.colab import files

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping
from keras import backend
from keras import layers
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedShuffleSplit
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
drive.mount('/content/gdrive')
drive_dir = "/content/gdrive/MyDrive/Teknik Informatika/Semester 7/TUGAS AKHIR/TA 2"

Mounted at /content/gdrive


## Feature Extraction

In [4]:
features_path = drive_dir + "/Eksperimen/Data"

In [5]:
feature_akustik = arff.load(features_path + '/eGeMAPS.arff')

In [6]:
def read_arff_data(data):
  row = next(data)
  row = next(data)
  list_data = []
  i = 0
  while row:
    if (i % 1000 == 0):
      print('Reading...', i, 'data already done.')
    row_data = []
    for j in range(len(row)):
      if (j != 0 and row[j] != ''):
        row_data.append(row[j])
    list_data.append(row_data)
    try :
      row = next(data)
      i = i + 1
    except :
      break
  return np.array(list_data)

In [7]:
feature_akustik = read_arff_data(feature_akustik)

Reading... 0 data already done.
Reading... 1000 data already done.
Reading... 2000 data already done.
Reading... 3000 data already done.
Reading... 4000 data already done.
Reading... 5000 data already done.
Reading... 6000 data already done.
Reading... 7000 data already done.
Reading... 8000 data already done.
Reading... 9000 data already done.
Reading... 10000 data already done.


In [8]:
print('Features Akustik', feature_akustik.shape)

Features Akustik (10822, 88)


In [9]:
processed_data = joblib.load(drive_dir + '/Eksperimen/Data/basic.pkl')

## Data Preparation

### Scaling Data Akustik

In [60]:
scaler = MinMaxScaler()
feature_akustik = scaler.fit_transform(feature_akustik)
feature_akustik.shape

(10822, 88)

### Construct Sequential Data 

In [61]:
def construct_sequential(id, data):
  sequential_datas = []
  new_data= []
  for i in range(len(id)):
    if id[i].endswith('001'):
      sequential_datas.append(new_data)
      new_data = []
    new_data.append(data[i])
  sequential_datas.append(new_data)

  return np.array(sequential_datas, dtype=object)[1:]

In [62]:
def construct_abstract(id, abstraksi):
  new_abstraksi = []
  for i in range(len(id)):
    if id[i].endswith('001'):
      new_abstraksi.append(abstraksi[i])

  return np.array(new_abstraksi, dtype=object)

In [63]:
id = processed_data['ID Ucapan']
sequential_features_akustik = construct_sequential(id, feature_akustik)
print(sequential_features_akustik.shape)

(2003,)


### Prepare Labels

In [64]:
encoder_ucapan = OneHotEncoder(sparse=False)
labels = np.array(processed_data['Label Emosi']).reshape(-1, 1)
labels = encoder_ucapan.fit_transform(labels)
labels = construct_sequential(id, labels)
labels.shape

(2003,)

In [65]:
abstraksi = construct_abstract(id, processed_data['Abstraksi Emosi'])
encoder_cakapan = OneHotEncoder(sparse=False)
abstraksi = abstraksi.reshape(-1, 1)
abstraksi = encoder_cakapan.fit_transform(abstraksi)
abstraksi.shape

(2003, 6)

### Split Train and Test Data

In [66]:
X = sequential_features_akustik
y_sequential = labels
y = abstraksi

In [67]:
X = tf.keras.preprocessing.sequence.pad_sequences(X, padding="post", dtype='float32')
y_sequential = tf.keras.preprocessing.sequence.pad_sequences(y_sequential, padding="post", dtype='float32')
y = tf.keras.preprocessing.sequence.pad_sequences(y, padding="post", dtype='float32')
print(X.shape, y.shape, y_sequential.shape)

(2003, 15, 88) (2003, 6) (2003, 15, 6)


In [68]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=42)
for train_index, test_index in sss.split(X, y):
  train_index, test_index = train_index, test_index

## Modelling Preparation

### Define Model

In [19]:
def get_contextual_model(shape, label_shape):
  model = Sequential()
  model.add(layers.Masking(mask_value=0, input_shape=shape))
  model.add(layers.LSTM(128, return_sequences=True))
  model.add(layers.Dropout(0.2))
  # model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dense(label_shape, activation='softmax'))
  return model

In [20]:
def get_classification_model(shape, label_shape):
  model = Sequential()
  model.add(layers.Masking(mask_value=0, input_shape=shape))
  model.add(layers.LSTM(128))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(label_shape, activation='softmax'))
  return model

### Define Training Parameter

In [21]:
batch_size = 32
epochs = 20
validation_split = 0.2
optimizer = 'adam'
loss = 'categorical_crossentropy'
metrics = ['accuracy']
callbacks = [EarlyStopping(monitor='val_loss', mode='min', min_delta=0.0001, patience=2, restore_best_weights=True)]

### Define Schema Experiment

In [22]:
def train_5fold_cv(X, y, shape, label_shape, return_sequences):

  loss_score = []
  accuracy_score = []  

  cv = KFold(n_splits=5, random_state=42, shuffle=True)

  for train_index, val_index in cv.split(X):
      
      X_train, X_val, y_train, y_val = X[train_index], X[val_index], y[train_index], y[val_index]
      
      model = None
      if return_sequences :
        model = get_contextual_model(shape, label_shape)
      else :
        model = get_classification_model(shape, label_shape)
      model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
      model.fit(X_train, y_train,
              validation_data=(X_val, y_val),
              epochs=epochs,
              batch_size=batch_size,
              callbacks=callbacks)
      
      l, a = model.evaluate(X_val, y_val)
      loss_score.append(l)
      accuracy_score.append(a)
  
  loss_score = np.round(np.mean(loss_score), 4)
  accuracy_score = np.round(np.mean(accuracy_score), 4)
  print("Loss Score : ", loss_score, "Accuracy Score : ", accuracy_score)
  return loss_score, accuracy_score

In [23]:
def train_stratified_random_sampling(X_train, y_train, X_test, y_test, shape, label_shape, return_sequences):

  model = None
  if return_sequences :
    model = get_contextual_model(shape, label_shape)
  else :
    model = get_classification_model(shape, label_shape)
  model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
  model.fit(X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks)
      
  loss_score, accuracy_score = model.evaluate(X_test, y_test)
  loss_score = np.round(np.mean(loss_score), 4)
  accuracy_score = np.round(np.mean(accuracy_score), 4)

  print("Loss Score : ", loss_score, "Accuracy Score : ", accuracy_score)
  
  return model, accuracy_score

## Modul 1 : Pembangunan Contextual Model

In [24]:
X_train, y_train = X[train_index], y_sequential[train_index]
X_test, y_test = X[test_index], y_sequential[test_index]
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1702, 15, 88) (1702, 15, 6)
(301, 15, 88) (301, 15, 6)


In [25]:
def run_contextual_model(X_train, y_train, X_test, y_test, n):
  model = None
  best_acc = 0
  for i in range(n):
    model, accuracy_score = train_stratified_random_sampling(X_train, y_train, X_test, y_test, X_train.shape[1:], y_train.shape[2], True)
    if best_acc < accuracy_score:
      best_acc = accuracy_score
      best_model = model
  return best_model

In [26]:
callbacks = [EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.0001, patience=3, restore_best_weights=True)]
contextual_model = run_contextual_model(X_train, y_train, X_test, y_test, 10)

Epoch 1/20
54/54 [==============================] - 24s 37ms/step - loss: 0.5545 - accuracy: 0.3521 - val_loss: 0.4923 - val_accuracy: 0.3429
Epoch 2/20
54/54 [==============================] - 0s 8ms/step - loss: 0.5060 - accuracy: 0.3842 - val_loss: 0.4871 - val_accuracy: 0.3429
Epoch 3/20
54/54 [==============================] - 0s 8ms/step - loss: 0.5125 - accuracy: 0.3663 - val_loss: 0.4856 - val_accuracy: 0.3429
Epoch 4/20
54/54 [==============================] - 0s 8ms/step - loss: 0.5209 - accuracy: 0.3589 - val_loss: 0.4978 - val_accuracy: 0.3531
Epoch 5/20
54/54 [==============================] - 0s 8ms/step - loss: 0.5083 - accuracy: 0.3671 - val_loss: 0.4780 - val_accuracy: 0.3665
Epoch 6/20
54/54 [==============================] - 0s 8ms/step - loss: 0.4979 - accuracy: 0.3739 - val_loss: 0.4820 - val_accuracy: 0.3448
Epoch 7/20
54/54 [==============================] - 0s 8ms/step - loss: 0.5169 - accuracy: 0.3678 - val_loss: 0.4776 - val_accuracy: 0.3672
Epoch 8/20
54/54 [

In [27]:
contextual_model.evaluate(X_test, y_test)

10/10 [==============================] - 0s 4ms/step - loss: 0.4733 - accuracy: 0.3966


[0.47328445315361023, 0.3965517282485962]

In [ ]:
contextual_model.save(drive_dir + "/Eksperimen/Model/model_akustik_bertahap_contextual.h5")

### Extract Contextual Features

In [69]:
contextual_model = keras.models.load_model(drive_dir + "/Eksperimen/Model/model_akustik_bertahap_contextual.h5")
contextual_model.evaluate(X[test_index], y_sequential[test_index])

10/10 [==============================] - 1s 4ms/step - loss: 0.4750 - accuracy: 0.3997


[0.4750272035598755, 0.39974457025527954]

In [70]:
def extract_contextual_features(model, idx_output, data):
  get_relu_output = backend.function([model.layers[0].input], [model.layers[idx_output].output])
  relu_output = get_relu_output([data])
  return np.array(relu_output[0])

In [71]:
contextual_features = extract_contextual_features(contextual_model, 3, X)
contextual_features.shape

(2003, 15, 128)

## Modul 2 : Pembangunan Classification Model

### 6 Kelas Emosi

In [72]:
X_train, y_train = contextual_features[train_index], y[train_index]
X_test, y_test = contextual_features[test_index], y[test_index]
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1702, 15, 128) (1702, 6)
(301, 15, 128) (301, 6)


In [59]:
def run_classification_model(X_train, y_train, X_test, y_test, n):
  model = None
  best_acc = 0
  for i in range(n):
    model, accuracy_score = train_stratified_random_sampling(X_train, y_train, X_test, y_test, X_train.shape[1:], y_train.shape[1], False)
    if best_acc < accuracy_score:
      best_acc = accuracy_score
      best_model = model
  return best_model

In [ ]:
classification_model = run_classification_model(X_train, y_train, X_test, y_test, 10)

In [73]:
models = []
for i in range(20):
  classification_model = run_classification_model(X_train, y_train, X_test, y_test, 1)
  pred = classification_model.predict(X_test)
  y_pred = encoder_cakapan.inverse_transform(pred)
  y_true = encoder_cakapan.inverse_transform(y_test)
  print(classification_report(y_true, y_pred, digits=4))
# print(confusion_matrix(y_true, y_pred)) 
  models.append(classification_model)

Epoch 1/20
54/54 [==============================] - 5s 28ms/step - loss: 1.5572 - accuracy: 0.3709 - val_loss: 1.4049 - val_accuracy: 0.3754
Epoch 2/20
54/54 [==============================] - 0s 7ms/step - loss: 1.4143 - accuracy: 0.3591 - val_loss: 1.3837 - val_accuracy: 0.3787
Epoch 3/20
54/54 [==============================] - 0s 7ms/step - loss: 1.4086 - accuracy: 0.3710 - val_loss: 1.3845 - val_accuracy: 0.3920
Epoch 4/20
54/54 [==============================] - 0s 7ms/step - loss: 1.3620 - accuracy: 0.3716 - val_loss: 1.3741 - val_accuracy: 0.4252
Epoch 5/20
54/54 [==============================] - 0s 7ms/step - loss: 1.3492 - accuracy: 0.3815 - val_loss: 1.3544 - val_accuracy: 0.4186
Epoch 6/20
54/54 [==============================] - 0s 7ms/step - loss: 1.3613 - accuracy: 0.4145 - val_loss: 1.3667 - val_accuracy: 0.3953
Epoch 7/20
10/10 [==============================] - 0s 4ms/step - loss: 1.3741 - accuracy: 0.4252
Loss Score :  1.3741 Accuracy Score :  0.4252


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       Jijik     0.0000    0.0000    0.0000         1
       Marah     0.6111    0.2619    0.3667        42
       Sedih     0.5294    0.1047    0.1748        86
      Senang     0.4060    0.9558    0.5699       113
       Takut     0.0000    0.0000    0.0000         9
    Terkejut     0.0000    0.0000    0.0000        50

    accuracy                         0.4252       301
   macro avg     0.2578    0.2204    0.1852       301
weighted avg     0.3890    0.4252    0.3151       301

Epoch 1/20
54/54 [==============================] - 33s 29ms/step - loss: 1.5577 - accuracy: 0.3208 - val_loss: 1.3852 - val_accuracy: 0.4020
Epoch 2/20
54/54 [==============================] - 0s 7ms/step - loss: 1.4223 - accuracy: 0.3478 - val_loss: 1.3746 - val_accuracy: 0.3688
Epoch 3/20
54/54 [==============================] - 0s 7ms/step - loss: 1.3715 - accuracy: 0.3773 - val_loss: 1.3846 - val_accuracy: 0.3754
Epoch 4/20
10/10 [=================

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       Jijik     0.0000    0.0000    0.0000         1
       Marah     0.8000    0.0952    0.1702        42
       Sedih     0.4286    0.1047    0.1682        86
      Senang     0.3927    0.9558    0.5567       113
       Takut     0.0000    0.0000    0.0000         9
    Terkejut     0.0000    0.0000    0.0000        50

    accuracy                         0.4020       301
   macro avg     0.2702    0.1926    0.1492       301
weighted avg     0.3815    0.4020    0.2808       301

Epoch 1/20
54/54 [==============================] - 5s 29ms/step - loss: 1.5465 - accuracy: 0.3122 - val_loss: 1.4277 - val_accuracy: 0.3787
Epoch 2/20
54/54 [==============================] - 0s 8ms/step - loss: 1.4083 - accuracy: 0.3689 - val_loss: 1.4163 - val_accuracy: 0.3322
Epoch 3/20
54/54 [==============================] - 0s 7ms/step - loss: 1.3920 - accuracy: 0.3546 - val_loss: 1.3672 - val_accuracy: 0.3821
Epoch 4/20
54/54 [==================

...

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       Jijik     0.0000    0.0000    0.0000         1
       Marah     0.4091    0.4286    0.4186        42
       Sedih     0.4030    0.3140    0.3529        86
      Senang     0.4305    0.5752    0.4924       113
       Takut     0.0000    0.0000    0.0000         9
    Terkejut     0.4103    0.3200    0.3596        50

    accuracy                         0.4186       301
   macro avg     0.2755    0.2730    0.2706       301
weighted avg     0.4020    0.4186    0.4038       301

Epoch 1/20
54/54 [==============================] - 5s 28ms/step - loss: 1.5551 - accuracy: 0.3478 - val_loss: 1.4006 - val_accuracy: 0.3920
Epoch 2/20
54/54 [==============================] - 0s 7ms/step - loss: 1.4062 - accuracy: 0.3477 - val_loss: 1.3791 - val_accuracy: 0.4053
Epoch 3/20
54/54 [==============================] - 0s 7ms/step - loss: 1.3665 - accuracy: 0.3812 - val_loss: 1.3717 - val_accuracy: 0.3887
Epoch 4/20
54/54 [==================

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       Jijik     0.0000    0.0000    0.0000         1
       Marah     0.4643    0.3095    0.3714        42
       Sedih     0.4737    0.1047    0.1714        86
      Senang     0.4127    0.9204    0.5699       113
       Takut     0.0000    0.0000    0.0000         9
    Terkejut     0.5000    0.0200    0.0385        50

    accuracy                         0.4219       301
   macro avg     0.3084    0.2258    0.1919       301
weighted avg     0.4381    0.4219    0.3211       301

Epoch 1/20
54/54 [==============================] - 6s 29ms/step - loss: 1.5559 - accuracy: 0.3495 - val_loss: 1.4082 - val_accuracy: 0.3754
Epoch 2/20
54/54 [==============================] - 0s 7ms/step - loss: 1.3907 - accuracy: 0.3997 - val_loss: 1.3899 - val_accuracy: 0.3289
Epoch 3/20
54/54 [==============================] - 0s 7ms/step - loss: 1.4017 - accuracy: 0.3646 - val_loss: 1.3655 - val_accuracy: 0.3987
Epoch 4/20
54/54 [==================

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [74]:
models[13].save(drive_dir + "/Eksperimen/Model/model_akustik_bertahap_classification_6.h5")

In [75]:
# classification_model2 = keras.models.load_model(drive_dir + "/Eksperimen/Model/model_akustik_bertahap_classification_6.h5")
classification_model2 = models[13]
pred = classification_model2.predict(X_test)
y_pred = encoder_cakapan.inverse_transform(pred)
y_true = encoder_cakapan.inverse_transform(y_test)
print(classification_report(y_true, y_pred, digits=4))
print(confusion_matrix(y_true, y_pred)) 

              precision    recall  f1-score   support

       Jijik     0.0000    0.0000    0.0000         1
       Marah     0.4000    0.4286    0.4138        42
       Sedih     0.3974    0.3605    0.3780        86
      Senang     0.4476    0.5664    0.5000       113
       Takut     0.0000    0.0000    0.0000         9
    Terkejut     0.4571    0.3200    0.3765        50

    accuracy                         0.4286       301
   macro avg     0.2837    0.2792    0.2781       301
weighted avg     0.4133    0.4286    0.4160       301

[[ 0  0  0  0  0  1]
 [ 0 18  9 11  0  4]
 [ 0  9 31 43  0  3]
 [ 0 11 27 64  0 11]
 [ 0  1  2  6  0  0]
 [ 0  6  9 19  0 16]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Delete Small Size Label Data

In [42]:
def get_label_idxs(list, label):
  idxs = []
  for i in range(len(list)):
    for l in label:
      if list[i] == l:
        idxs.append(i)
  return np.array(idxs)

In [43]:
def delete_elArr_by_idx(arr, idx):
  new_arr = []
  for i in range(len(arr)):
    if i not in idx:
      new_arr.append(arr[i])
  return np.array(new_arr)

In [44]:
dec_abstraksi = encoder_cakapan.inverse_transform(abstraksi)
del_idxs = get_label_idxs(dec_abstraksi, ["Jijik", "Takut"])
updated_features = delete_elArr_by_idx(contextual_features, del_idxs)
updated_labels = delete_elArr_by_idx(dec_abstraksi, del_idxs)
print(updated_features.shape, updated_labels.shape)

(1936, 15, 128) (1936, 1)


In [45]:
encoder_cakapan_2 = OneHotEncoder(sparse=False)
updated_labels = updated_labels.reshape(-1, 1)
updated_labels = encoder_cakapan_2.fit_transform(updated_labels)
updated_labels.shape

(1936, 4)

In [46]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=42)
for train_index, test_index in sss.split(updated_features, updated_labels):
  train_index, test_index = train_index, test_index

In [47]:
X_train, y_train = updated_features[train_index], updated_labels[train_index]
X_test, y_test = updated_features[test_index], updated_labels[test_index]
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1645, 15, 128) (1645, 4)
(291, 15, 128) (291, 4)


In [ ]:
classification_model = run_classification_model(X_train, y_train, X_test, y_test, 10)

Epoch 1/20
52/52 [==============================] - 6s 47ms/step - loss: 1.2876 - accuracy: 0.3832 - val_loss: 1.2157 - val_accuracy: 0.4158
Epoch 2/20
52/52 [==============================] - 1s 21ms/step - loss: 1.2167 - accuracy: 0.4134 - val_loss: 1.2101 - val_accuracy: 0.4536
Epoch 3/20
52/52 [==============================] - 1s 22ms/step - loss: 1.2252 - accuracy: 0.4294 - val_loss: 1.2021 - val_accuracy: 0.4467
Epoch 4/20
52/52 [==============================] - 1s 22ms/step - loss: 1.2191 - accuracy: 0.4072 - val_loss: 1.1952 - val_accuracy: 0.4502
Epoch 5/20
52/52 [==============================] - 1s 21ms/step - loss: 1.2245 - accuracy: 0.4090 - val_loss: 1.1953 - val_accuracy: 0.4536
Epoch 6/20
52/52 [==============================] - 1s 21ms/step - loss: 1.2095 - accuracy: 0.4294 - val_loss: 1.1931 - val_accuracy: 0.4227
Epoch 7/20
52/52 [==============================] - 1s 21ms/step - loss: 1.1906 - accuracy: 0.4407 - val_loss: 1.1891 - val_accuracy: 0.4708
Epoch 8/20
52

In [49]:
models = []
for i in range(20):
  classification_model = run_classification_model(X_train, y_train, X_test, y_test, 1)
  pred = classification_model.predict(X_test)
  y_pred = encoder_cakapan_2.inverse_transform(pred)
  y_true = encoder_cakapan_2.inverse_transform(y_test)
  print(classification_report(y_true, y_pred, digits=4))
# print(confusion_matrix(y_true, y_pred))
  models.append(classification_model)

Epoch 1/20
52/52 [==============================] - 5s 30ms/step - loss: 1.3146 - accuracy: 0.3985 - val_loss: 1.2435 - val_accuracy: 0.4021
Epoch 2/20
52/52 [==============================] - 0s 7ms/step - loss: 1.2459 - accuracy: 0.4017 - val_loss: 1.2352 - val_accuracy: 0.4055
Epoch 3/20
52/52 [==============================] - 0s 8ms/step - loss: 1.2353 - accuracy: 0.4124 - val_loss: 1.2301 - val_accuracy: 0.4261
Epoch 4/20
52/52 [==============================] - 0s 8ms/step - loss: 1.2339 - accuracy: 0.3958 - val_loss: 1.2287 - val_accuracy: 0.4124
Epoch 5/20
52/52 [==============================] - 0s 7ms/step - loss: 1.2206 - accuracy: 0.4165 - val_loss: 1.2236 - val_accuracy: 0.4055
Epoch 6/20
10/10 [==============================] - 0s 3ms/step - loss: 1.2301 - accuracy: 0.4261
Loss Score :  1.2301 Accuracy Score :  0.4261
              precision    recall  f1-score   support

       Marah     0.4545    0.3571    0.4000        42
       Sedih     0.4167    0.1163    0.1818   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       Marah     0.3947    0.3571    0.3750        42
       Sedih     0.4444    0.0930    0.1538        86
      Senang     0.4213    0.8761    0.5690       113
    Terkejut     0.0000    0.0000    0.0000        50

    accuracy                         0.4192       291
   macro avg     0.3151    0.3316    0.2745       291
weighted avg     0.3519    0.4192    0.3205       291

Epoch 1/20
52/52 [==============================] - 5s 28ms/step - loss: 1.3112 - accuracy: 0.3569 - val_loss: 1.2496 - val_accuracy: 0.3986
Epoch 2/20
52/52 [==============================] - 0s 7ms/step - loss: 1.2507 - accuracy: 0.4120 - val_loss: 1.2406 - val_accuracy: 0.4055
Epoch 3/20
52/52 [==============================] - 0s 7ms/step - loss: 1.2547 - accuracy: 0.3942 - val_loss: 1.2403 - val_accuracy: 0.3883
Epoch 4/20
52/52 [==============================] - 0s 7ms/step - loss: 1.2487 - accuracy: 0.4029 - val_loss: 1.2400 - val_accuracy: 0.3814
Epoc

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       Marah     0.3889    0.1667    0.2333        42
       Sedih     0.4000    0.1628    0.2314        86
      Senang     0.4076    0.8584    0.5527       113
    Terkejut     0.0000    0.0000    0.0000        50

    accuracy                         0.4055       291
   macro avg     0.2991    0.2970    0.2544       291
weighted avg     0.3326    0.4055    0.3167       291

Epoch 1/20
52/52 [==============================] - 5s 30ms/step - loss: 1.3253 - accuracy: 0.3868 - val_loss: 1.2722 - val_accuracy: 0.3746
Epoch 2/20
52/52 [==============================] - 0s 7ms/step - loss: 1.2638 - accuracy: 0.4064 - val_loss: 1.2414 - val_accuracy: 0.4158
Epoch 3/20
52/52 [==============================] - 0s 7ms/step - loss: 1.2448 - accuracy: 0.4069 - val_loss: 1.2534 - val_accuracy: 0.3918
Epoch 4/20
52/52 [==============================] - 0s 8ms/step - loss: 1.2636 - accuracy: 0.3948 - val_loss: 1.2305 - val_accuracy: 0.4124
Epoc

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       Marah     0.4516    0.3333    0.3836        42
       Sedih     0.0000    0.0000    0.0000        86
      Senang     0.4109    0.9381    0.5714       113
    Terkejut     1.0000    0.0400    0.0769        50

    accuracy                         0.4192       291
   macro avg     0.4656    0.3278    0.2580       291
weighted avg     0.3965    0.4192    0.2905       291

Epoch 1/20
52/52 [==============================] - 6s 29ms/step - loss: 1.3167 - accuracy: 0.3808 - val_loss: 1.2421 - val_accuracy: 0.4021
Epoch 2/20
52/52 [==============================] - 0s 7ms/step - loss: 1.2451 - accuracy: 0.4186 - val_loss: 1.2446 - val_accuracy: 0.4124
Epoch 3/20
52/52 [==============================] - 0s 7ms/step - loss: 1.2426 - accuracy: 0.4023 - val_loss: 1.2325 - val_accuracy: 0.3986
Epoch 4/20
52/52 [==============================] - 0s 7ms/step - loss: 1.2352 - accuracy: 0.3984 - val_loss: 1.2276 - val_accuracy: 0.4227
Epoc

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [55]:
models[12].save(drive_dir + "/Eksperimen/Model/model_akustik_bertahap_classification_4.h5")

In [56]:
classification_model = keras.models.load_model(drive_dir + "/Eksperimen/Model/model_akustik_bertahap_classification_4.h5")
pred = classification_model.predict(X_test)
y_pred = encoder_cakapan_2.inverse_transform(pred)
y_true = encoder_cakapan_2.inverse_transform(y_test)
print(classification_report(y_true, y_pred, digits=4))
print(confusion_matrix(y_true, y_pred)) 

              precision    recall  f1-score   support

       Marah     0.4667    0.3333    0.3889        42
       Sedih     0.4409    0.4767    0.4581        86
      Senang     0.4552    0.5841    0.5116       113
    Terkejut     0.4348    0.2000    0.2740        50

    accuracy                         0.4502       291
   macro avg     0.4494    0.3985    0.4081       291
weighted avg     0.4491    0.4502    0.4373       291

[[14 11 15  2]
 [ 6 41 37  2]
 [ 7 31 66  9]
 [ 3 10 27 10]]
